In [1625]:
## Подключаем необходимые библиотеки
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('seaborn-darkgrid')
import pandas as pd
import yfinance as yf
import matplotlib
from pandas_datareader import data as pdr
from yahoo_fin.stock_info import get_data
import yahoo_fin.stock_info as si
from pandas_datareader import data as web
import datetime
from nsepy import get_history
from nsepython import *
import logging
urllib3_logger = logging.getLogger('urllib3') # Обработчик сообщений, чтобы они не выводились при сборе данных о тикерах
urllib3_logger.setLevel(logging.CRITICAL)
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import cufflinks as cf
cf.go_offline()

DEBUG:matplotlib.pyplot:Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [1626]:
# Задаем ETF на страну. 
ticker = 'IXC'
start = datetime.datetime(2010,1,1)
end = datetime.datetime.today().date() # сегодняшняя дата, чтобы не менять вручную.
# Получаем данные из Yahoo. Именно этот способ позволяет получить данные с тикерами в столбцах.
f = pdr.DataReader(ticker, 'yahoo', start, end)

In [1627]:
data = pd.DataFrame(f["Adj Close"]).set_axis(['Price'], axis=1, inplace=False)

In [1628]:
# Методика SMA
# data['SMA10'] = data['Price'].rolling(window=10).mean()
# data['SMA20'] = data['Price'].rolling(window=20).mean()
# sma10 = data['SMA10'] > data['SMA20']
# price = data['Price'] > data['SMA20']
# data['Signal'] = np.where(sma10 & price,1,0)

In [1632]:
# Методика EMA
data['SMA10'] = pd.Series.ewm(data['Price'], span=10).mean()
data['SMA20'] = pd.Series.ewm(data['Price'], span=20).mean()
sma10 = data['SMA10'] > data['SMA20']
price = data['Price'] > data['SMA20']
data['Signal'] = np.where(sma10 & price,1,0)

In [1633]:
data[['Price', 'SMA10', 'SMA20', 'Signal']].iplot(secondary_y = ['Signal'], title = "Price & MA", xTitle = "Data", yTitle = "Price")

In [1634]:
data['Price_returns'] = data['Price'].pct_change()
data['Strategy_return'] = data['Price_returns'] * data['Signal'].shift()
data['Price_returns']

Date
2009-12-31         NaN
2010-01-04    0.030830
2010-01-05    0.005438
2010-01-06    0.008653
2010-01-07   -0.002949
                ...   
2021-06-14    0.002207
2021-06-15    0.016520
2021-06-16   -0.007223
2021-06-17   -0.029465
2021-06-18   -0.029985
Name: Price_returns, Length: 2886, dtype: float64

In [1635]:
data['Cum_Buy_Hold_Return'] = (data['Price_returns'] + 1).cumprod()
data['Cum_Strategy_Return'] = (data['Strategy_return'] + 1).cumprod()

In [1636]:
data[['Cum_Buy_Hold_Return', 'Cum_Strategy_Return']].iplot(title = "BH VS MA", xTitle = "Data", yTitle = "Price")

In [1637]:
running_max = np.maximum.accumulate(data['Cum_Buy_Hold_Return'].dropna())
running_max2 = np.maximum.accumulate(data['Cum_Strategy_Return'].dropna())
# Ensure the value never drops below 1
running_max[running_max < 1] = 1
running_max2[running_max2 < 1] = 1
# Calculate the percentage drawdown
drawdown = (data['Cum_Buy_Hold_Return'])/running_max - 1
drawdown2 = (data['Cum_Strategy_Return'])/running_max2 - 1
max_dd = drawdown.min()*100
max_dd2 = drawdown2.min()*100

In [1638]:
print('Return from Buy and Hold: ', round(100*(data['Cum_Buy_Hold_Return'][-1]-1),2), '%')
print('Return from strategy: ', round(100*(data['Cum_Strategy_Return'][-1]-1),2), '%')
print('The Sharpe ratio B&H strategy ', round(np.mean(data['Price_returns'])/np.std(data['Price_returns'])*(252**0.5),2), '%' )
print('The Sharpe ratio strategy ', round(np.mean(data['Strategy_return'])/np.std(data['Strategy_return'])*(252**0.5),2), '%' )
print('The maximum hold-buy drawdown is', "%.2f%%" % max_dd)
print('The maximum strategy drawdown is', "%.2f%%" % max_dd2)

Return from Buy and Hold:  6.26 %
Return from strategy:  51.39 %
The Sharpe ratio B&H strategy  0.15 %
The Sharpe ratio strategy  0.34 %
The maximum hold-buy drawdown is -67.88%
The maximum strategy drawdown is -25.30%


In [1639]:
print(round(100*(data['Cum_Buy_Hold_Return'][-1]-1),2), '%')
print(round(100*(data['Cum_Strategy_Return'][-1]-1),2), '%')
print(round(np.mean(data['Price_returns'])/np.std(data['Price_returns'])*(252**0.5),2), '%' )
print(round(np.mean(data['Strategy_return'])/np.std(data['Strategy_return'])*(252**0.5),2), '%' )
print("%.2f%%" % max_dd)
print("%.2f%%" % max_dd2)

6.26 %
51.39 %
0.15 %
0.34 %
-67.88%
-25.30%
